In [60]:
import os
import tempfile
import shutil
import xml.etree.ElementTree as ET
from datetime import datetime
from typing import Tuple, Dict, List

from tqdm.notebook import tqdm

from common import *

In [36]:
def load_workflow(path: str) -> Graph:
    graph = get_graph()
    graph.parse(path, format='turtle')
    return graph

In [37]:
def get_workflow_steps(graph: Graph) -> List[URIRef]:
    steps = list(graph.subjects(RDF.type, dtbox.Step))
    return steps

In [38]:
def get_base_node_config():
    root = ET.Element('config', {'xmlns': 'http://www.knime.org/2008/09/XMLConfig',
                                 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 'xsi:schemaLocation': 'http://www.knime.org/2008/09/XMLConfig http://www.knime.org/XMLConfig_2008_09.xsd',
                                 'key': 'settings.xml',
                                 })

    ET.SubElement(root, 'entry', {'key': 'node_file', 'type': 'xstring', 'value': 'settings.xml'})
    ET.SubElement(root, 'entry', {'key': 'customDescription', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'state', 'type': 'xstring', 'value': 'CONFIGURED'})
    ET.SubElement(root, 'entry', {'key': 'hasContent', 'type': 'xboolean', 'value': 'false'})
    ET.SubElement(root, 'entry', {'key': 'isInactive', 'type': 'xboolean', 'value': 'false'})
    ET.SubElement(root, 'config', {'key': 'factory_settings'})
    ET.SubElement(root, 'entry', {'key': 'node-feature-name', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry',
                  {'key': 'node-feature-symbolic-name', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'node-feature-vendor', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'node-feature-version', 'type': 'xstring', 'value': '0.0.0'})

    return ET.ElementTree(root)

In [39]:
def get_step_component_implementation(ontology: Graph, workflow_graph: Graph, step: URIRef) -> Tuple[URIRef, URIRef]:
    component = next(workflow_graph.objects(step, dtbox.runs, True))
    implementation = next(ontology.objects(component, dtbox.hasImplementation, True))
    return component, implementation

In [46]:
def get_knime_properties(ontology: Graph, implementation: URIRef) -> Dict[str, str]:
    results = {}
    for p, o in ontology.predicate_objects(implementation):
        if p.fragment.startswith('knime'):
            results[p.fragment[6:]] = o.value
    return results

In [71]:
def create_step_file(ontology: Graph, workflow_graph: Graph, step: URIRef, folder, iterator: int) -> str:
    tree = get_base_node_config()
    root = tree.getroot()
    component, implementation = get_step_component_implementation(ontology, workflow_graph, step)
    properties = get_knime_properties(ontology, implementation)

    for key, value in properties.items():
        ET.SubElement(root, 'entry', {'key': key, 'type': 'xstring', 'value': value})

    path_name = properties["node-name"].replace('(', '_').replace(')', '_')
    subfolder_name = f'{path_name} (#{iterator})'
    subfolder = os.path.join(folder, subfolder_name)
    os.mkdir(subfolder)
    tree.write(os.path.join(subfolder, 'settings.xml'), encoding='utf-8', xml_declaration=True)
    return subfolder_name


In [10]:
def create_workflow_metadata_file(workflow_graph: Graph, folder: str) -> None:
    author = 'Diviloper'
    date = datetime.today().strftime('%d/%m/%Y')
    workflow_name = next(workflow_graph.subjects(RDF.type, dtbox.Workflow, True)).fragment
    comments = f'''
    This workflow was automatically created from the logical workflow {workflow_name}
'''
    template = f'''
<?xml version="1.0" encoding="UTF-8"?><KNIMEMetaInfo nrOfElements="3">
    <element form="text" read-only="false" name="Author">{author}</element>
    <element form="date" name="Creation Date" read-only="false">{date}</element>
    <element form="multiline" name="Comments" read-only="false">{comments}</element>
</KNIMEMetaInfo>
'''
    with open(os.path.join(folder, 'workflowset.meta'), 'w') as f:
        f.write(template)

In [ ]:
def get_nodes_config(step_paths: List[str]) -> ET.Element:
    root = ET.Element('config', {'key': 'nodes'})
    for i, step in enumerate(step_paths):
        node_cofig = ET.SubElement(root, 'config', {'key': f'node_{i}'})
        ET.SubElement(node_cofig, 'entry', {'key': 'id', 'type': 'xint', 'value': i})
        ET.SubElement(node_cofig, 'entry', {'key': 'node_settings_file', 'type': 'xstring',
                                            'value': f'{step}/settings.xml'})
        ET.SubElement(node_cofig, 'entry', {'key': 'node_is_meta', 'type': 'xboolean', 'value': 'false'})
        ET.SubElement(node_cofig, 'entry', {'key': 'node_type', 'type': 'xstring', 'value': 'NativeNode'})
        ET.SubElement(node_cofig, 'entry', {'key': 'ui_classname', 'type': 'xstring',
                                            'value': 'org.knime.core.node.workflow.NodeUIInformation'})

    return root

In [ ]:
def get_workflow_connections(workflow_graph: Graph) -> List[Tuple[URIRef, URIRef, URIRef, URIRef]]:
    query = f'''
    PREFIX dtbox: <{dtbox}>
    SELECT ?source ?destination ?sourcePort ?destinationPort
    WHERE {{
        ?source a dtbox:Step .
                dtbox:followedBy ?destination .
                dtbox:hasOutput ?output .
        ?output dtbox:has_position ?sourcePort ;
                dtbox:hasData ?link .
        ?destination a dtbox:Step ;
                    dtbox:hasInput ?input .
        ?input dtbox:has_position ?destinationPort ;
                dtbox:hasData ?link .
    }}
    '''
    results = workflow_graph.query(query).bindings
    return [(r['source'], r['destination'], r['sourcePort'], r['destinationPort']) for r in results]

In [ ]:
def get_connections_config(workflow_graph: Graph, steps: List[URIRef]) -> ET.Element:
    root = ET.Element('config', {'key': 'connections'})
    connections = get_workflow_connections(workflow_graph)
    for i, (source, destination, source_port, destination_port) in enumerate(connections):
        connection_config = ET.SubElement(root, 'config', {'key': f'connection_{i}'})
        ET.SubElement(connection_config, 'entry', {'key': 'sourceID', 'type': 'xint', 'value': steps.index(source)})
        ET.SubElement(connection_config, 'entry', {'key': 'sourcePort', 'type': 'xint', 'value': source_port + 1})
        ET.SubElement(connection_config, 'entry', {'key': 'destID', 'type': 'xint', 'value': steps.index(destination)})
        ET.SubElement(connection_config, 'entry', {'key': 'destPort', 'type': 'xint', 'value': destination_port + 1})
        ET.SubElement(connection_config, 'entry', {'key': 'ui_classname', 'type': 'xstring',
                                                   'value': 'org.knime.core.node.workflow.ConnectionUIInformation'})
    return root

In [ ]:
def get_author_config() -> ET.Element:
    root = ET.Element('config', {'key': 'authorInformation'})
    ET.SubElement(root, 'entry', {'key': 'authored-by', 'type': 'xstring', 'value': 'Diviloper'})
    ET.SubElement(root, 'entry', {'key': 'authored-when', 'type': 'xstring', 'value': datetime.today().strftime('%Y-%m-%d %H:%M:%S %z')})
    ET.SubElement(root, 'entry', {'key': 'lastEdited-by', 'type': 'xstring', 'value': 'Diviloper'})
    ET.SubElement(root, 'entry', {'key': 'lastEdited-when', 'type': 'xstring', 'value': datetime.today().strftime('%Y-%m-%d %H:%M:%S %z')})
    return root

In [ ]:
def create_workflow_file(ontology: Graph, workflow_graph: Graph, steps: List[URIRef], step_paths: List[str], folder: str) -> None:
    node_config = get_nodes_config(step_paths)
    connections_config = get_connections_config(workflow_graph, steps)
    author_config = get_author_config()

    root = ET.Element('config', {'xmlns': 'http://www.knime.org/2008/09/XMLConfig',
                                 'xmlns:xsi': 'http://www.w3.org/2001/XMLSchema-instance',
                                 'xsi:schemaLocation': 'http://www.knime.org/2008/09/XMLConfig http://www.knime.org/XMLConfig_2008_09.xsd',
                                 'key': 'workflow.knime'})

    ET.SubElement(root, 'entry', {'key': 'created_by', 'type': 'xstring', 'value': 'Diviloper'})
    ET.SubElement(root, 'entry', {'key': 'created_by_nightly', 'type': 'xboolean', 'value': 'false'})
    # ET.SubElement(root, 'entry', {'key': 'version', 'type': 'xstring', 'value': '4.1.0'})
    ET.SubElement(root, 'entry', {'key': 'name', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'customDescription', 'type': 'xstring', 'isnull': 'true', 'value': ''})
    ET.SubElement(root, 'entry', {'key': 'state', 'type': 'xstring', 'value': 'IDLE'})
    ET.SubElement(root, 'config', {'key': 'workflow_credentials'})

    root.append(node_config)
    root.append(connections_config)
    root.append(author_config)

    tree = ET.ElementTree(root)
    tree.write(os.path.join(folder, 'workflow.knime'), encoding='UTF-8', xml_declaration=True)


In [68]:
def translate_graph(ontology: Graph, source_path: str, destination_path: str) -> None:
    print('Creating new workflow')

    print('\tCreating temp folder: ', end='')
    temp_folder = tempfile.mkdtemp()
    print(temp_folder)

    print('\tLoading workflow:', end=' ')
    graph = load_workflow(source_path)
    print(next(graph.subjects(RDF.type, dtbox.Workflow, True)).fragment)

    print('\tCreating workflow metadata file')
    create_workflow_metadata_file(graph, temp_folder)

    print('\tBuilding steps')
    steps = get_workflow_steps(graph)
    step_paths = []
    for i, step in enumerate(steps):
        step_paths.append(create_step_file(ontology, graph, step, temp_folder, i))

    print('\tCreating workflow file')
    create_workflow_file(ontology, graph, steps, step_paths, temp_folder)

    print('\tCreating zip file')
    shutil.make_archive(destination_path, 'zip', temp_folder)
    os.rename(destination_path + '.zip', destination_path)

    print('\tRemoving temp folder')
    shutil.rmtree(temp_folder)
    print('Done')
    print('-' * 50)


In [69]:
def translate_graph_folder(ontology: Graph, source_folder: str, destination_folder: str) -> None:
    assert os.path.exists(destination_folder)
    assert os.path.exists(source_folder)

    workflows = [f for f in os.listdir(source_folder) if f.endswith('.ttl')]
    for workflow in tqdm(workflows):
        source_path = os.path.join(source_folder, workflow)
        destination_path = os.path.join(destination_folder, workflow[:-4] + '.knwf')
        translate_graph(ontology, source_path, destination_path)

In [51]:
ontology = get_ontology_graph()
source_path = '../pipeline_generator/workflows/2023-07-15 21-40-38/workflow_0_DescriptionIntent_b949aa3c_e664_43eb_ae32_004cac74fd24.ttl'
destination_path = './workflows/2023-07-15 20-27-10/workflow_10_DescriptionIntent_4762afff_2235_4c9e_b3ac_6876f5b634ed.knwf'

temp_folder = tempfile.mkdtemp()
print(temp_folder)
graph = load_workflow(source_path)
steps = get_workflow_steps(graph)
print(steps)
create_step_file(ontology, graph, steps[0], temp_folder, 0)

In [70]:
translate_graph_folder(ontology, '../pipeline_generator/workflows/2023-07-15 23-10-28',
                       './workflows/2023-07-15 23-10-28')

  0%|          | 0/72 [00:00<?, ?it/s]

Creating new workflow
	Creating temp folder: C:\Users\Victor\AppData\Local\Temp\tmpfbckqcbr
	Loading workflow: workflow_0_DescriptionIntent_2232f589_594f_4b1c_b460_abb517e6b7a8
	Building steps
	Creating zip file
	Removing temp folder
Done
--------------------------------------------------
Creating new workflow
	Creating temp folder: C:\Users\Victor\AppData\Local\Temp\tmp8od0fpv2
	Loading workflow: workflow_10_DescriptionIntent_d12c1481_b9aa_4013_8884_64fa5998200c
	Building steps
	Creating zip file
	Removing temp folder
Done
--------------------------------------------------
Creating new workflow
	Creating temp folder: C:\Users\Victor\AppData\Local\Temp\tmpl7huaaq0
	Loading workflow: workflow_11_DescriptionIntent_24b62ee5_587d_4892_a437_9dc9144fc91c
	Building steps
	Creating zip file
	Removing temp folder
Done
--------------------------------------------------
Creating new workflow
	Creating temp folder: C:\Users\Victor\AppData\Local\Temp\tmp_q9ous1k
	Loading workflow: workflow_12_Descr